In [1]:
########################################################################
# File       : Read_and_Display_Images_using_Widgets_and_Napari.ipynb
# Version    : 0.1
# Author     : czsrh
# Date       : 12.11.2019
# Insitution : Carl Zeiss Microscopy GmbH
#
# Disclaimer: Just for testing - Use at your own risk.
# Feedback or Improvements are welcome.
########################################################################

***Import Libaries to run this notebook***

* [PyPi - czifile](https://pypi.org/project/czifile/)

Thanks to Christoph Gohlke. For details and the source code please see [czifile.py](https://www.lfd.uci.edu/~gohlke/code/czifile.py.html)

* [PyPi - apeer-ometiff-library](https://pypi.org/project/apeer-ometiff-library/)

It uses parts of python-bioformats library: [PyPi - python-bioformats](https://pypi.org/project/python-bioformats/).

More information on the source code can be found on the APEER GitHub project page: [GitHub - apeer-ometiff-library](https://github.com/apeer-micro/apeer-ometiff-library)

* [PyPi - napari](https://pypi.org/project/napari/)

Very new, fast and cool multi-dimensional array viewer. For more details see the prject page [GitHub - napari](https://github.com/napari/napari)

* imagefileutils

This is collection of of usefule function to read CZI and OME-TFF image files. This tool uses many of the modules mentioned above

***

Another important modules

* [PyPi - xmltodict](https://pypi.org/project/xmltodict/)

This is turns an XML, e.g. the OME-XML or CZI metadata into an "easy-to-use" python dictionary.

In [2]:

# this can be used to switch on/off warnings
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

# import the libraries mentioned above
from apeer_ometiff_library import io, processing, omexmlClass
import czifile as zis
import xmltodict
import os
import numpy as np
import ipywidgets as widgets
import napari
import imgfileutils as imf
import xml.etree.ElementTree as ET

In [3]:
# define your testfiles here

testfolder = r'C:\Users\m1srh\Documents\GitHub\ipy_notebooks\Read_OMETIFF_CZI\testdata'

imgdict = {1: os.path.join(testfolder, r'CellDivision_T=10_Z=15_CH=2_DCV_small_green.ome.tiff'),
           2: os.path.join(testfolder, r'CellDivision_T=10_Z=15_CH=2_DCV_small_red.ome.tiff'),
           3: os.path.join(testfolder, r'CellDivision_T=10_Z=15_CH=2_DCV_small.ome.tiff'),
           4: os.path.join(testfolder, r'CellDivision_T=10_Z=15_CH=2_DCV_small_green.czi'),
           5: os.path.join(testfolder, r'CellDivision_T=10_Z=15_CH=2_DCV_small_red.czi'),
           6: os.path.join(testfolder, r'CellDivision_T=10_Z=15_CH=2_DCV_small.czi'),
           7: r'/datadisk1/tuxedo/testpictures/Testdata_Zeiss/em/FIB_Stack.czi'
}

filename = imgdict[7]

In [4]:
# To read the CZI metadata one can use the czifile functionality

# get czi object using the filename
czi = zis.CziFile(filename)

# get the metadata and close the czi file object
mdczi = czi.metadata()
czi.close()

# parse the XML into a dictionary
metadatadict_czi = xmltodict.parse(mdczi)

In [5]:
# Sometime it can be very usefule to save the CZI metadate as an actual XML file.
# This can be done quite easily by converting the metadata string into an XML tree.
# This tree object can be directly as an XML file

# define the new filename for the XML to be created later
xmlfile = filename.replace('.czi', '_CZI_MetaData.xml')

# get the element tree
tree = ET.ElementTree(ET.fromstring(mdczi))

# write xml to disk
tree.write(xmlfile, encoding='utf-8', method='xml')

print('Write special CZI XML metainformation for: ', xmlfile)

Write special CZI XML metainformation for:  /datadisk1/tuxedo/testpictures/Testdata_Zeiss/em/FIB_Stack_CZI_MetaData.xml


In [6]:
# or much shorter
xmlczi = imf.writexml_czi(filename)
print(xmlczi)

/datadisk1/tuxedo/testpictures/Testdata_Zeiss/em/FIB_Stack_CZI_MetaData.xml


### Remark

As one can clearly see there are a lot of metadata not all of them are needed for every workflow.

In [7]:
# read metadata and array differently for OME-TIFF or CZI data
if filename.lower().endswith('.ome.tiff') or filename.lower().endswith('.ome.tif'):
    
    # Return value is an array of order (T, Z, C, X, Y)
    (array, omexml) = io.read_ometiff(filename)
    metadata = imf.get_metadata(filename, series=0)
    
if filename.lower().endswith('.czi'):

    # get the array and the metadata
    array, metadata = imf.get_array_czi(filename)

Key not found: 'AcquisitionDateAndTime'


In [8]:
# outout the shape of the returned numpy array

# shape of numpy array
print('Array Shape: ', array.shape)

# dimension order from metadata
print('Dimension Order (BioFormats) : ', metadata['DimOrder BF Array'])

# shape and dimension entry from CZI file as returned by czifile.py
print('CZI Array Shape : ', metadata['Shape'])
print('CZI Dimension Entry : ', metadata['Axes'])

Array Shape:  (44, 1536, 2048)
Dimension Order (BioFormats) :  None
CZI Array Shape :  (44, 1536, 2048, 1)
CZI Dimension Entry :  ZYX0


In [9]:
# show dimensions and scaling
print('SizeT : ', metadata['SizeT'])
print('SizeZ : ', metadata['SizeZ'])
print('SizeC : ', metadata['SizeC'])
print('SizeX : ', metadata['SizeX'])
print('SizeY : ', metadata['SizeY'])
print('XScale: ', metadata['XScale'])
print('YScale: ', metadata['YScale'])
print('ZScale: ', metadata['ZScale'])

SizeT :  1
SizeZ :  44
SizeC :  1
SizeX :  2048
SizeY :  1536
XScale:  1.0
YScale:  1.0
ZScale:  0.02


In [10]:
# show all the metadata
for key, value in metadata.items():
    # print all key-value pairs for the dictionary
    print(key, ' : ', value)

Directory  :  /datadisk1/tuxedo/testpictures/Testdata_Zeiss/em
Filename  :  FIB_Stack.czi
Extension  :  czi
ImageType  :  czi
Name  :  None
AcqDate  :  None
TotalSeries  :  None
SizeX  :  2048
SizeY  :  1536
SizeZ  :  44
SizeC  :  1
SizeT  :  1
Sizes BF  :  None
DimOrder BF  :  None
DimOrder BF Array  :  None
DimOrder CZI  :  {'B': -1, 'S': -1, 'T': -1, 'C': -1, 'Z': 0, 'Y': 1, 'X': 2, '0': 3}
Axes  :  ZYX0
Shape  :  (44, 1536, 2048, 1)
isRGB  :  None
ObjNA  :  None
ObjMag  :  None
ObjID  :  Objective:1
ObjName  :  None
ObjImmersion  :  None
XScale  :  1.0
YScale  :  1.0
ZScale  :  0.02
XScaleUnit  :  None
YScaleUnit  :  None
ZScaleUnit  :  None
DetectorModel  :  None
DetectorName  :  None
DetectorID  :  None
InstrumentID  :  None
Channels  :  ['0']
ImageIDs  :  []
NumPy.dtype  :  uint8
Experiment  :  None
PixelType  :  Gray8
SizeM  :  1
SizeB  :  1
SizeS  :  1
SW-Name  :  ZEN 3.0 (blue edition)
SW-Version  :  2.6.77.0000
TubelensMag  :  None
ObjNominalMag  :  None


In [11]:
# display data using ipy widgets
if metadata['Extension'] == 'ome.tiff':
    ui, out = imf.create_ipyviewer_ome_tiff(array, metadata)
if metadata['Extension'] == 'czi':
    ui, out = imf.create_ipyviewer_czi(array, metadata)

display(ui, out)

25 255


Output()

In [ ]:
# try to configure napari automatiaclly based on metadata
imf.show_napari(array, metadata)

In [ ]:
# configure napari viewer manually - check array shape and dimensions order carefully 
    
# get the scalefactors
scalefactors = imf.get_scalefactor(metadata)
print(scalefactors)

array = np.squeeze(array, axis=(0, 1))

viewer = napari.Viewer()
# add every channel as a single layer
for ch in range(metadata['SizeC']):
    chname = metadata['Channels'][ch]
    viewer.add_image(array[ch, :, :, :], name=chname, scale=(1, scalefactors['zx'], 1, 1))

jupyter nbconvert MIAP_Read_and_Display_Images.ipynb --to slides --post serve